In [205]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, FloatSlider, IntSlider


In [ ]:
sns.set(style="darkgrid")

def plot_sinus(f=5.0, fs=50):
    t_end = 1
    t = np.linspace(0, t_end, 1000)
    y = np.sin(2 * np.pi * f * t)
    t_samples = np.arange(0, t_end, 1/fs)
    y_samples = np.sin(2 * np.pi * f * t_samples)

    plt.figure(figsize=(10, 4))
    plt.plot(t, y, label='Sygnał ciągły', color='grey')
    plt.scatter(t_samples, y_samples, color='#5C0120', label='Próbki')
    plt.title(f'Sygnał sinusoidalny: f = {f} Hz, fs = {fs} Hz')
    plt.xlabel('Czas [s]')
    plt.ylabel('Amplituda')
    plt.grid(True)
    plt.legend()
    plt.ylim(-1.2, 1.2)
    plt.show()

interact(plot_sinus,
         f=FloatSlider(value=5, min=1, max=20, step=0.5, description='f [Hz]'),
         fs=IntSlider(value=15, min=5, max=100, step=10, description='fs [Hz]'))


interactive(children=(FloatSlider(value=5.0, description='f [Hz]', max=20.0, min=1.0, step=0.5), IntSlider(val…

<function __main__.plot_sinus(f=5.0, fs=50)>

In [207]:
def piecewise_linear_interpolation(t_cont : int, t_samples : list, y_samples : list):
    '''
    Proste liczenie funkcji liniowych (y = ax + b) pomiędzy punktami i sklejenie ich w np.piecewise
    Input:
        t_count - liczba punktów,
        t_samples - wartości czasowe dla punktów
        y_samples - wartości y dla punktów
        
    Output:
        sygnal dyskretny z wartościami pomiędzy punktami obliczonymi za pomoca funkcji liniowych pomiędzy punktami
    '''
    conditions = []
    functions = []

    for i in range(len(t_samples) - 1):
        x0, x1 = t_samples[i], t_samples[i + 1]
        y0, y1 = y_samples[i], y_samples[i + 1]
        m = (y1 - y0) / (x1 - x0)
        b = y0 - m * x0
        conditions.append((t_cont >= x0) & (t_cont < x1))
        functions.append(lambda t, m=m, b=b: m * t + b)

    conditions.append(t_cont >= t_samples[-1])
    functions.append(lambda t: y_samples[-1])

    return np.piecewise(t_cont, conditions, functions)

def plot_interpolation(f=5.0, fs=20):
    t_end = 1
    t = np.linspace(0, t_end, 1000)
    y = np.sin(2 * np.pi * f * t)
    t_samples = np.arange(0, t_end + 1/fs, 1/fs)
    y_samples = np.sin(2 * np.pi * f * t_samples)
    y_interp = piecewise_linear_interpolation(t, t_samples, y_samples)
    error = y - y_interp
    
    plt.figure(figsize=(10, 4))
    plt.plot(t, y, label='Sygnał oryginalny', color='grey')
    plt.plot(t, y_interp, '--', label='Interpolacja', color='#7F1734')
    plt.scatter(t_samples, y_samples, color='#5C0120', label='Próbki')
    plt.title(f'Interpolacja liniowa: f = {f} Hz, fs = {fs} Hz')
    plt.xlabel('Czas [s]')
    plt.ylabel('Amplituda')
    plt.grid(True)
    plt.legend()
    plt.ylim(-1.2, 1.2)
    plt.show()

    plt.figure(figsize=(10, 3))
    plt.plot(t, error, color='#006666')
    plt.title('Błąd interpolacji (oryginał - interpolacja)')
    plt.xlabel('Czas [s]')
    plt.ylabel('Błąd')
    plt.grid(True)
    plt.show()

interact(plot_interpolation,
         f=FloatSlider(value=5.0, min=1.0, max=20.0, step=0.5, description='f [Hz]'),
         fs=IntSlider(value=15, min=5, max=100, step=5, description='fs [Hz]'))


interactive(children=(FloatSlider(value=5.0, description='f [Hz]', max=20.0, min=1.0, step=0.5), IntSlider(val…

<function __main__.plot_interpolation(f=5.0, fs=20)>

In [208]:
def whittaker_shannon_interpolation(t, t_samples, x_samples, T):  
    y_interp = np.zeros_like(t)
    for n in range(len(x_samples)):
        y_interp += x_samples[n] * np.sinc((t - t_samples[n]) / T) # sinc(x) = sin(πx) / (πx)
    return y_interp

def plot_interpolation(f=5.0, fs=20):
    t_end = 1
    T = 1 / fs
    t_cont = np.linspace(0, t_end, 1000)
    y_cont = np.sin(2 * np.pi * f * t_cont)
    t_samples = np.arange(0, t_end + T, T)
    x_samples = np.sin(2 * np.pi * f * t_samples)
    y_interp = whittaker_shannon_interpolation(t_cont, t_samples, x_samples, T)
    error = y_cont - y_interp

    plt.figure(figsize=(10, 4))
    plt.plot(t_cont, y_cont, label='Sygnał oryginalny', color='grey')
    plt.plot(t_cont, y_interp, '--', label='Interpolacja Whittaker–Shannon', color='#7F1734')
    plt.scatter(t_samples, x_samples, color='#5C0120', label='Próbki')
    plt.title(f'Interpolacja Whittakera–Shannona: f = {f} Hz, fs = {fs} Hz')
    plt.xlabel('Czas [s]')
    plt.ylabel('Amplituda')
    plt.ylim(-1.2, 1.2)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(10, 3))
    plt.plot(t_cont, error, color='#006666')
    plt.title('Błąd interpolacji (oryginał - interpolacja)')
    plt.xlabel('Czas [s]')
    plt.ylabel('Błąd')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

interact(plot_interpolation,
         f=FloatSlider(value=5.0, min=1.0, max=20.0, step=0.5, description='f [Hz]'),
         fs=IntSlider(value=15, min=5, max=100, step=5, description='fs [Hz]'))


interactive(children=(FloatSlider(value=5.0, description='f [Hz]', max=20.0, min=1.0, step=0.5), IntSlider(val…

<function __main__.plot_interpolation(f=5.0, fs=20)>